<a href="https://colab.research.google.com/github/monicaneli/Deteccao-em-imagens-rede-Yolo/blob/main/Reconhecimento_Facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção e Reconhecimento Facial

O presente projeto tem como objetivo desenvolver um sistema automatizado de detecção e reconhecimento de faces, aplicando-o especificamente para identificar os personagens principais da série *The Big Bang Theory*. A solução combina técnicas modernas de visão computacional e aprendizado profundo para realizar duas tarefas principais: detecção de faces em imagens ou vídeos e reconhecimento de identidades específicas.

### 1. Metodologia

**Detecção de faces:**
Para localizar rostos nas imagens, foi utilizada a rede MTCNN (Multi-task Cascaded Convolutional Networks).

**Reconhecimento facial:** FaceNet e um classificador leve (KNN ou SVM).

### 2. Base de Dados

Para o reconhecimento dos personagens, foi construída uma base de 12 imagens por personagem, contendo apenas o rosto, garantindo um treinamento eficiente mesmo com um número reduzido de exemplos. Técnicas de *data augmentation* (giro, flip, ajustes de brilho) foram aplicadas para aumentar virtualmente o conjunto de dados e melhorar a robustez do modelo.


### 3. Tecnologias e Ferramentas

* **Python** com bibliotecas: OpenCV, MTCNN, keras-facenet, scikit-learn.
* **Redes neurais profundas**: MTCNN para detecção e FaceNet para reconhecimento.
* **Classificador leve**: KNN, para mapear embeddings em identidades de personagens.
* **Data augmentation**: aumento de base para robustez.





In [1]:
# Imports e parâmetros
import os
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [2]:
!wget -O the-big-bang-theory-3.jpg 'https://observatoriodocinema.com.br/wp-content/plugins/seox-image-magick/imagick_convert.php?width=1869&height=1402&format=.jpg&quality=91&imagick=/wp-content/uploads/2022/03/the-big-bang-theory-3.jpg'
!wget -O Amy_TBBT.jpg  'https://aventurasnahistoria.com.br/wp-content/uploads/curiosidades/amy_fowler_mayim_bialik.jpg'

--2025-09-15 21:33:13--  https://observatoriodocinema.com.br/wp-content/plugins/seox-image-magick/imagick_convert.php?width=1869&height=1402&format=.jpg&quality=91&imagick=/wp-content/uploads/2022/03/the-big-bang-theory-3.jpg
Resolving observatoriodocinema.com.br (observatoriodocinema.com.br)... 104.21.70.51, 172.67.220.1, 2606:4700:3037::6815:4633, ...
Connecting to observatoriodocinema.com.br (observatoriodocinema.com.br)|104.21.70.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpg]
Saving to: ‘the-big-bang-theory-3.jpg’

the-big-bang-theory     [   <=>              ] 559.53K   844KB/s    in 0.7s    

2025-09-15 21:33:15 (844 KB/s) - ‘the-big-bang-theory-3.jpg’ saved [572955]

--2025-09-15 21:33:15--  https://aventurasnahistoria.com.br/wp-content/uploads/curiosidades/amy_fowler_mayim_bialik.jpg
Resolving aventurasnahistoria.com.br (aventurasnahistoria.com.br)... 172.67.213.242, 104.21.16.157, 2606:4700:3036::6815:109d, ...
Connecting t

In [3]:
# Banco de faces para reconhecimento
dataset_path = "/content/TBBT_Faces"

# Dataset Big Bang Theory
!wget -O TBBT.zip "https://github.com/monicaneli/Reconhecimento-Facial/raw/main/TBBT_Faces.zip"
!unzip -q TBBT.zip

--2025-09-15 21:33:15--  https://github.com/monicaneli/Reconhecimento-Facial/raw/main/TBBT_Faces.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/monicaneli/Reconhecimento-Facial/main/TBBT_Faces.zip [following]
--2025-09-15 21:33:15--  https://raw.githubusercontent.com/monicaneli/Reconhecimento-Facial/main/TBBT_Faces.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1004611 (981K) [application/zip]
Saving to: ‘TBBT.zip’

TBBT.zip            100%[===================>] 981.07K  --.-KB/s    in 0.06s   

2025-09-15 21:33:16 (16.9 MB/s) - ‘TBBT.zip’ saved [1004611/1004611]



In [12]:
!pip install mtcnn
!pip install keras-facenet
!pip install lz4

In [13]:
# Imports
import cv2
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from keras_facenet import FaceNet

## Detecção de faces com MTCNN

Para localizar rostos nas imagens, foi utilizada a rede MTCNN (Multi-task Cascaded Convolutional Networks).

Esta rede é projetada para detectar rostos com alta precisão em diferentes posições, ângulos e condições de iluminação. Ela realiza a detecção em três estágios cascata, refinando progressivamente as localizações de face e estimando pontos-chave faciais (olhos, nariz, boca), permitindo um recorte preciso dos rostos.


In [14]:
# Rede MTCNN (Multi-task Cascaded Convolutional Networks)
detector = MTCNN()

ValueError: LZ4 is not installed. Install it with pip: https://python-lz4.readthedocs.io/

In [ ]:
# Teste de detecção de faces
img = cv2.imread("/content/Amy_TBBT.jpg")
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

faces = detector.detect_faces(rgb)

for f in faces:
    x, y, w, h = f['box']
    x, y = max(0,x), max(0,y)  # Garantir que não seja negativo
    face_crop = rgb[y:y+h, x:x+w]

    # Salvar face recortada
    cv2.imwrite("face_detected.jpg", cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR))

    # Criar uma cópia para desenhar o retângulo
    img_overlay = img.copy()

    # Desenhar retângulo verde ao redor da face
    cv2.rectangle(img_overlay, (x, y), (x+w, y+h), color=(0,255,0), thickness=3)

    # Mostrar imagem com retângulo
    plt.figure(figsize=(6,6))
    plt.imshow(cv2.cvtColor(img_overlay, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()



## Reconhcimento Facial

Após a detecção, cada face é processada pela rede FaceNet, que gera embeddings (vetores de características) para cada rosto.

O FaceNet é uma rede profunda pré-treinada que mapeia faces em um espaço de alta dimensão, de forma que rostos da mesma pessoa fiquem próximos entre si, enquanto rostos de pessoas diferentes fiquem distantes.

Esses embeddings são então utilizados para treinar um classificador leve (KNN ou SVM), capaz de reconhecer cada personagem a partir de um pequeno conjunto de imagens de referência.

In [ ]:
# Carregar FaceNet
embedder = FaceNet()

# Função para extrair embeddings de uma face
def get_embedding(face_img):
    # Redimensionar para 160x160, que é o padrão do FaceNet
    face = cv2.resize(face_img, (160, 160))
    face = face.astype("float32")
    face = np.expand_dims(face, axis=0)
    return embedder.embeddings(face)[0]


In [ ]:
#Comparar face detectada com banco
from numpy.linalg import norm

def recognize_face(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return "Erro: imagem não carregada"
    emb = get_embedding(img)
    pred = knn.predict([emb])[0]
    proba = knn.predict_proba([emb])[0]
    return le.inverse_transform([pred])[0], np.max(proba)

def recognize_face2(face_img, threshold=0.8):
    emb = get_embedding(face_img)
    best_match = None
    best_score = float("inf")

    for person, embeddings in known_faces.items():
        for ref_emb in embeddings:
            score = norm(emb - ref_emb)
            if score < best_score:
                best_score = score
                best_match = person

    if best_score < threshold:
        return best_match, best_score
    else:
        return "Desconhecido", best_score


In [ ]:
# Pasta de rostos extraídos do dataset
embedder = FaceNet()
X, y = [], []

def augment_image(img):
    """Aplica aumentos simples em uma imagem"""
    aug_imgs = []

    # 1. Flip horizontal
    aug_imgs.append(cv2.flip(img, 1))

    # 2. Rotação pequena
    angle = random.choice([-10, -5, 5, 10])
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    aug_imgs.append(cv2.warpAffine(img, M, (w, h)))

    # 3. Alterar brilho
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[...,1] = hsv[...,1]*random.uniform(0.8,1.2)
    hsv[...,2] = hsv[...,2]*random.uniform(0.7,1.3)
    hsv[hsv > 255] = 255
    aug_imgs.append(cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2BGR))

    return aug_imgs

# Preparando o dataset
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    if not os.path.isdir(person_path):
        continue

    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, (160, 160))

        # Embedding da imagem original
        X.append(embedder.embeddings([img])[0])
        y.append(person)

        # Augmentation → Embeddings das variações
        for aug in augment_image(img):
            aug = cv2.resize(aug, (160, 160))
            X.append(embedder.embeddings([aug])[0])
            y.append(person)

X = np.array(X)
y = np.array(y)

print("Total de embeddings após augmentation:", X.shape, "Labels:", len(y))

In [ ]:
# Transformar rótulos em inteiros
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Treinar um KNN simples
knn = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
knn.fit(X, y_enc)

print("Classes:", le.classes_)


In [ ]:
# Cria o classificador SVM com kernel linear
svm = SVC(kernel='linear', probability=True)  # probability=True permite prever confiança
svm.fit(X, y_enc)

print("Classes aprendidas pelo SVM:", le.classes_)

In [ ]:
def recognize_face_svm(image_path):
    img = cv2.imread(image_path)  # Read the image from the path
    if img is None:
        return "Erro: imagem não carregada", 0.0 # Return error if image not loaded
    face_resized = cv2.resize(img, (160, 160))
    emb = embedder.embeddings([face_resized])[0]
    pred = svm.predict([emb])[0]
    proba = np.max(svm.predict_proba([emb])[0])
    label = le.inverse_transform([pred])[0]
    return label, proba

# Teste
test_img = '/content/Amy_TBBT.jpg'
label, score = recognize_face_svm(test_img)
print("Reconhecido como:", label, "com confiança:", score)

In [ ]:
# Exemplo: testar com uma imagem da Amy
test_img = '/content/Amy_TBBT.jpg'
label, score = recognize_face(test_img)
print("Reconhecido como:", label, " (confiança:", score, ")")

## Reconhecimeno de múltiplas faces em uma imagem


In [ ]:
def recognize_faces_in_frame(frame_path, knn, embedder, le):
    # Carregar frame
    img = cv2.imread(frame_path)
    if img is None:
        return "Erro ao carregar imagem"
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detectar faces
    results = detector.detect_faces(rgb)
    print("Faces detectadas:", len(results))

    for res in results:
        x, y, w, h = res['box']
        face = rgb[y:y+h, x:x+w]
        face = cv2.resize(face, (160, 160))

        # Extrair embedding
        emb = embedder.embeddings([face])[0]

        # Predição
        pred = knn.predict([emb])[0]
        proba = knn.predict_proba([emb])[0]
        label = le.inverse_transform([pred])[0]
        score = np.max(proba)

        # Desenhar bounding box e nome
        cv2.rectangle(rgb, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(rgb, f"{label} ({score:.2f})", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    # Mostrar imagem final
    plt.figure(figsize=(10,8))
    plt.imshow(rgb)
    plt.axis("off")
    plt.show()

In [ ]:
frame_path = "/content/the-big-bang-theory-3.jpg"  # vários personagens
recognize_faces_in_frame(frame_path, knn, embedder, le)

## Conclusão

O sistema proposto combina detecção de faces de alta precisão com reconhecimento baseado em embeddings, permitindo identificar os personagens de *The Big Bang Theory* de forma automática e eficiente.

A escolha de MTCNN e FaceNet garante robustez frente a variações de pose, expressão e iluminação, oferecendo uma solução prática para aplicações de reconhecimento facial em séries, vídeos ou imagens estáticas.


## Referências

[1] YOLOv4 Object Detection on Webcam In Google Colab por [Jack Wotherspoon](https://github.com/jackwotherspoon)

[2] Bootcamp BairesDev - Machine Learning Training pela DIO